In [2]:
import pandas as pd
import numpy as np
import seaborn as sns

In [3]:
df = pd.read_csv("all_matches.csv")


c:\users\samriddhi singh\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
remove_columns = ['season', 'start_date', 'striker', 'non_striker', 'bowler', 'byes', 'legbyes', 'noballs',
                  'penalty', 'wicket_type', 'player_dismissed', 'other_wicket_type', 'other_player_dismissed', 'wides']
df.drop(labels = remove_columns, axis = 1, inplace = True)

df = df.fillna(0)
df['score'] = (df["runs_off_bat"] + df["extras"]).astype("int")
df = df[["match_id", "venue", "innings", "batting_team", "bowling_team", "ball", "score"]]
df = df[df.ball <= 6.0]

In [5]:
current_teams = ['Kolkata Knight Riders', 'Chennai Super Kings', 'Rajasthan Royals', 'Mumbai Indians', 
                 'Kings XI Punjab', 'Royal Challengers Bangalore', 'Delhi Daredevils', 'Sunrisers Hyderabad']

current_venues = ['M Chinnaswamy Stadium', 'Eden Gardens', 'MA Chidambaram Stadium', 'Wankhede Stadium', 
                  'Arun Jaitley Stadium']

df = df[(df['batting_team'].isin(current_teams)) &(df['bowling_team'].isin(current_teams))]
df = df[(df['venue'].isin(current_venues))]

In [6]:
df['score_6'] = df.groupby(['match_id', 'innings'])['score'].transform('sum')

In [7]:
dummies = pd.get_dummies(df.venue)
merged = pd.concat([df,dummies],axis='columns')
df = merged.drop(['venue'],axis='columns')

dummies2 = pd.get_dummies(df.batting_team)
merged2 = pd.concat([df,dummies2],axis='columns')
df = merged2.drop(['batting_team'],axis='columns')

dummies3 = pd.get_dummies(df.bowling_team)
merged3 = pd.concat([df,dummies3],axis='columns')
df = merged3.drop(['bowling_team'],axis='columns')

In [8]:
df = df.drop(['match_id'],axis='columns')

In [9]:
X = df.drop('score_6',axis=1)
y = df['score_6']

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.9, random_state=0)

In [11]:
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators = 1000, random_state = 42)
regressor.fit(X_train, y_train)

RandomForestRegressor(n_estimators=1000, random_state=42)

In [12]:
y_pred = regressor.predict(X_test)

In [13]:
df=pd.DataFrame({'Actual':y_test, 'Predicted':y_pred})
df

,Actual,Predicted
172280,45,50.641750
73015,50,48.788233
43233,24,26.390964
50665,35,51.281700
133598,51,54.574667
...,...,...
134137,59,40.085429
105693,52,49.270783
108501,100,59.403000
146,26,39.827000


In [14]:
from sklearn import metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
from sklearn.metrics import r2_score
print('R Squared Error:', np.sqrt(metrics.r2_score(y_test, y_pred)))

Mean Absolute Error: 9.168938739853555
Mean Squared Error: 153.3159370841245
Root Mean Squared Error: 12.382081290482812
R Squared Error: 0.2801182620257563


In [15]:
# Calculate the absolute errors
errors = abs(y_pred - y_test)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')


Mean Absolute Error: 9.17 degrees.
Accuracy: 77.06 %.
